# POI Download

## POI from Overture

In [1]:
import duckdb

# Connect to DuckDB
con = duckdb.connect()

# Install and load necessary extensions
con.install_extension('httpfs')
con.install_extension('spatial')
con.load_extension('httpfs')
con.load_extension('spatial')

# Define the URL and bbox
url = 'https://data.source.coop/cholmes/overture/places-geoparquet-country/FR.parquet'
minx, miny, maxx, maxy = 2.321001, 48.837379, 2.373657, 48.870248

# Define the bbox polygon WKT
bbox_wkt = f'POLYGON(({minx} {miny}, {maxx} {miny}, {maxx} {maxy}, {minx} {maxy}, {minx} {miny}))'


In [2]:
schema_query = f"DESCRIBE SELECT * FROM read_parquet('{url}');"
schema = con.execute(schema_query).fetchdf()  # Or .fetchall() for a list of tuples
print(schema)

    column_name                                        column_type null   key  \
0            id                                            VARCHAR  YES  None   
1    updatetime                                            VARCHAR  YES  None   
2       version                                            INTEGER  YES  None   
3         names  STRUCT(key_value STRUCT("key" VARCHAR, "value"...  YES  None   
4    categories  STRUCT(alternate STRUCT(list VARCHAR[]), main ...  YES  None   
5    confidence                                             DOUBLE  YES  None   
6      websites                             STRUCT(list VARCHAR[])  YES  None   
7       socials                             STRUCT(list VARCHAR[])  YES  None   
8        emails                                            INTEGER  YES  None   
9        phones                             STRUCT(list VARCHAR[])  YES  None   
10        brand  STRUCT("names" STRUCT(key_value STRUCT("key" V...  YES  None   
11    addresses  STRUCT(list

In [3]:
# Query to filter and create GeoJSON
query = f"""
SELECT json_object(
    'type', 'FeatureCollection',
    'features', json_group_array(
        json_object(
            'type', 'Feature',
            'geometry', ST_AsGeoJSON(geometry)::json,
            'properties', json_object(
                'id', id,
                'confidence', confidence,
                'names', to_json(names),
                'categories', to_json(categories),
                'brand', to_json(brand),
                'addresses', to_json(addresses)
            )
        )
    )
) AS geojson
FROM (
    SELECT *
    FROM read_parquet('{url}')
    WHERE bbox.minx <= {maxx} AND bbox.maxx >= {minx} AND bbox.miny <= {maxy} AND bbox.maxy >= {miny}
    AND ST_Intersects(geometry, ST_GeomFromText('{bbox_wkt}'))
)
"""

# Execute the query and fetch the GeoJSON string
geojson_str = con.execute(query).fetchone()[0]

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [4]:
# Export to local file
with open('places.geojson', 'w') as f:
    f.write(geojson_str)

print("GeoJSON exported to places.geojson")

GeoJSON exported to places.geojson


NameError: name 'places' is not defined